In [31]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [32]:
# !pip install wordcloud
from wordcloud import WordCloud

In [33]:
import warnings
warnings.filterwarnings("ignore")

In [34]:
data = pd.read_excel("отзывы за лето.xls")

In [35]:
data.head(5)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [36]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
# !pip install pymorphy2
from pymorphy2 import MorphAnalyzer
from string import punctuation

In [38]:
russian_stopwords = stopwords.words("russian")

In [39]:
exclude = set(punctuation)
morpher = MorphAnalyzer()

In [40]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = (data['Rating'] > 3)*1

In [41]:
data['target'] = data['target'].astype(int)
data.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,в целое удобноной приложениеиз минус хотеть сл...,1
2,5,Отлично все,2017-08-14,отлично всё,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать на 1 работа антивирус далёкий н...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро,1


In [42]:
from sklearn.model_selection import train_test_split

In [43]:
data['target'].value_counts()

1    16724
0     3024
Name: target, dtype: int64

In [44]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

In [45]:
from sklearn.feature_extraction.text import HashingVectorizer

In [46]:
count_vect = HashingVectorizer(n_features=200).fit(X_train.values)

In [47]:
xtrain = count_vect.transform(X_train)
xtest = count_vect.transform(X_test)

In [48]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [49]:
num_classes = 2
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

In [92]:
X_train.shape

(15798,)

In [93]:
max_words = 200
max_len = 1

In [94]:
# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [95]:
model = Sequential()
model.add(Embedding(input_dim=200, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(2))
model.add(Activation('softmax'))

ValueError: ignored

In [89]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [90]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

In [91]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20


ValueError: ignored

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)